In [ ]:
import pandas as pd
import numpy as np
import cooler
from hic2cool import hic2cool_convert

In [ ]:
#Convert the HiC to cool file for futher operation. The resolution can be different in other datasets; here we use 10k as an example.
hic2cool_convert('HiChip_FoxP3_Treg_R1.hic', 'HiChip1.cool', 10000)
hic2cool_convert('HiChip_FoxP3_Treg_R2.hic', 'HiChip2.cool', 10000)

In [ ]:
#Convert the cool file to txt so that we can load them into pandas dataframe.
!cooler dump --join HiChip1.cool > HiChip1.txt
!cooler dump --join HiChip2.cool > HiChip2.txt
!cooler dump --join PLACseq.cool > PLACseq.txt

In [ ]:
#Read the TnG and NTnG bin files.
mm10TnG=pd.read_csv("SaDi_TnGBins_mm10.bed",sep='\t', names=['Chr', 'Start', 'End'],header=None)
mm10NTnG=pd.read_csv("SaDi_NTnGBins_mm10.bed",sep='\t', names=['Chr', 'Start', 'End'],header=None)
mm9TnG=pd.read_csv("SaDi_TnGBins_mm9.bed",sep='\t', names=['Chr', 'Start', 'End'],header=None)
mm9NTnG=pd.read_csv("SaDi_NTnGBins_mm9.bed",sep='\t', names=['Chr', 'Start', 'End'],header=None)

In [ ]:
#Read the converted cooler txt files into pandas dataframes.
HiChip1=pd.read_csv("HiChip1.txt", sep='\t', names=['ChrA', 'StartA', 'EndA','ChrB', 'StartB', 'EndB', "Ncontact"],header=None)
HiChip2=pd.read_csv("HiChip2.txt", sep='\t', names=['ChrA', 'StartA', 'EndA','ChrB', 'StartB', 'EndB', "Ncontact"],header=None)
PLACseq=pd.read_csv("PLACseq.txt", sep='\t', names=['ChrA', 'StartA', 'EndA','ChrB', 'StartB', 'EndB', "Ncontact"],header=None)

In [ ]:
#This function does the below three things:
#1. It takes the input of cool files and load into pandas dataframes. Each bin is given a unique name by its Chr, start and end.
#2. It takes the input of TnG or NTnG bin files, in either mm9 or mm10 genome. The HiChips are in mm10 genome, while PLACseq is in mm9.
#3. Saves the TnG/NTnG filtered cooler dataframes into csv files.
#4. A minimum of 2 is used to filter out low contact frequencies. You can change this if you want.
def Mean_Ncontact(coolname, dfA, dfB, filename):
    dfcool=pd.read_csv(f"{coolname}.txt", sep='\t', names=['ChrA', 'StartA', 'EndA','ChrB', 'StartB', 'EndB', "Ncontact"],header=None)
    dfcool["BinNameA"]=dfcool['ChrA'] + "_" + dfcool["StartA"].astype(str) + "to" + dfcool["EndA"].astype(str)
    dfcool["BinNameB"]=dfcool['ChrB'] + "_" + dfcool["StartB"].astype(str) + "to" + dfcool["EndB"].astype(str)
    dfA["BinNameA"]=dfA['Chr'] + "_" + dfA["Start"].astype(str) + "to" + dfA["End"].astype(str)
    dfB["BinNameB"]=dfB['Chr'] + "_" + dfB["Start"].astype(str) + "to" + dfB["End"].astype(str)
    dfcoolA = dfcool[dfcool.set_index(['BinNameA']).index.isin(dfA.set_index(['BinNameA']).index)]
    dfcoolAB = dfcoolA[dfcoolA.set_index(['BinNameB']).index.isin(dfB.set_index(['BinNameB']).index)]
    #A filter of 2 is used below.
    dfcoolAB=dfcoolAB.loc[dfcoolAB['Ncontact'] > 2]
    dfcoolAB = dfcoolAB[dfcoolAB['BinNameA'] != dfcoolAB['BinNameB']]
    #The next line calculates average if cool input is merged from multiple replicates
    dfcoolAB["Ncontact"]=dfcoolAB["Ncontact"]/2
    print("Mean Ncontact: ", dfcoolAB['Ncontact'].mean())
    dfcoolAB2=dfcoolAB.drop(columns=['BinNameA', 'BinNameB'])
    dfcoolAB3=dfcoolAB2.sort_values(by='Ncontact', ascending=False)
    dfcoolAB3.to_csv(f'{filename}.csv', index=False)
    del dfcool, dfcoolA, dfcoolAB, dfcoolAB2, dfcoolAB3
    #return dfcoolAB

In [ ]:
#Here we get the TnG-TnG, TnG-NTnG and NTnG-NTnG contact numbers and save them automatically to csv files.
Mean_Ncontact("HiChip1", mm10TnG, mm10TnG, "HiChip1_2TnG")
Mean_Ncontact("HiChip1", mm10TnG, mm10NTnG, "HiChip1_TnGNTnG")
Mean_Ncontact("HiChip1", mm10NTnG, mm10NTnG, "HiChip1_2NTnG")
Mean_Ncontact("HiChip1", mm10NTnG, mm10TnG, "HiChip1_NTnGTnG")
Mean_Ncontact("HiChip2", mm10TnG, mm10TnG, "HiChip2_2TnG")
Mean_Ncontact("HiChip2", mm10TnG, mm10NTnG, "HiChip2_TnGNTnG")
Mean_Ncontact("HiChip2", mm10NTnG, mm10NTnG, "HiChip2_2NTnG")
Mean_Ncontact("HiChip2", mm10NTnG, mm10TnG, "HiChip2_NTnGTnG")
Mean_Ncontact("PLACseq", mm9TnG, mm9TnG, "PLACseq_2TnG")
Mean_Ncontact("PLACseq", mm9TnG, mm9NTnG, "PLACseq_TnGNTnG")
Mean_Ncontact("PLACseq", mm9NTnG, mm9NTnG, "PLACseq_2NTnG")
Mean_Ncontact("PLACseq", mm9NTnG, mm9TnG, "PLACseq_NTnGTnG")

In [ ]:
#This function takes the above csv files, return the contact pairs as indexes so we can concat them later.
def Contactpairs(filename, pairname, samplename):
    dfcool=pd.read_csv(f"{filename}.csv")
    dfcool["BinNameA"]= dfcool['ChrA'] + "_" + dfcool["StartA"].astype(str) + "to" + dfcool["EndA"].astype(str)
    dfcool["BinNameB"]= dfcool['ChrB'] + "_" + dfcool["StartB"].astype(str) + "to" + dfcool["EndB"].astype(str)
    dfcool['Contact_pair'] = dfcool["BinNameA"] + "and" + dfcool["BinNameB"]
    #dfcool['Contact_group']= pairname
    dfcool=dfcool.drop(columns=['ChrA', 'StartA', 'EndA', 'ChrB', 'StartB', 'EndB', "BinNameA", "BinNameB"])
    dfcool=dfcool.rename(columns={"Ncontact": f"{samplename}"})
    dfcool=dfcool.set_index(["Contact_pair"])
    return dfcool

In [ ]:
#Processing all the HiChip samples here and save them into csv files:
#All of the csv files generated were uploaded in the "expected_output" folder in this GitHub repo.
HiChip1_2TnG=Contactpairs("HiChip1_2TnG", "2TnG", "HiChip1")
HiChip1_2NTnG=Contactpairs("HiChip1_2NTnG", "2NTnG", "HiChip1")
HiChip1_TnGNTnG=Contactpairs("HiChip1_TnGNTnG", "TnGNTnG", "HiChip1")
HiChip2_2TnG=Contactpairs("HiChip2_2TnG", "2TnG", "HiChip2")
HiChip2_2NTnG=Contactpairs("HiChip2_2NTnG", "2NTnG", "HiChip2")
HiChip2_TnGNTnG=Contactpairs("HiChip2_TnGNTnG", "TnGNTnG", "HiChip2")
HiChip_2TnG=pd.concat([HiChip1_2TnG, HiChip2_2TnG], axis=1).fillna(0)
HiChip_2TnG["Contact_group_HiChip"]="2TnG"
HiChip_2NTnG=pd.concat([HiChip1_2NTnG, HiChip2_2NTnG], axis=1).fillna(0)
HiChip_2NTnG["Contact_group_HiChip"]="2NTnG"
HiChip_TnGNTnG=pd.concat([HiChip1_TnGNTnG, HiChip2_TnGNTnG], axis=1).fillna(0)
HiChip_TnGNTnG["Contact_group_HiChip"]="TnGNTnG"
HiChip=pd.concat([HiChip_2TnG, HiChip_2NTnG,HiChip_TnGNTnG], axis=0).fillna(0)

In [ ]:
#Process all the PLACseq samples here and save them into csv files:
#All of the csv files generated were uploaded in the "expected_output" folder in this GitHub repo.
PLACseq_2TnG=Contactpairs("PLACseq_2TnG", "2TnG", "PLACseq")
PLACseq_2NTnG=Contactpairs("PLACseq_2NTnG", "2NTnG", "PLACseq")
PLACseq_TnGNTnG=Contactpairs("PLACseq_TnGNTnG", "TnGNTnG", "PLACseq")
PLACseq_2TnG["Contact_group_PLACseq"]="2TnG"
PLACseq_2NTnG["Contact_group_PLACseq"]="2NTnG"
PLACseq_TnGNTnG["Contact_group_PLACseq"]="TnGNTnG"
AllTPLACseq=pd.concat([PLACseq_2TnG, PLACseq_2NTnG, PLACseq_TnGNTnG], axis=0).fillna(0)